In [1]:
import           Control.Monad.Trans.State.Strict
import qualified Data.IntMap.Strict  as M
import qualified Data.Vector         as V

import           Data.Foldable (for_)
import           Data.Functor (void)

In [2]:
graph = M.fromList
    [ (1, [2])
    , (2, [3])
    , (3, [1])
    , (4, [2, 3, 5])
    , (5, [4, 6])
    , (6, [7, 3])
    , (7, [6])
    , (8, [5, 7, 8])
    ]

In [3]:
data TState = TState
    { index    :: Int
    , stack    :: [Int]
    , stackSet :: V.Vector Bool
    , stackTop :: Int
    , indices  :: V.Vector (Maybe Int)
    , lowlinks :: V.Vector (Maybe Int)
    , output   :: [[Int]]
    } deriving (Eq, Show)
  
data Event
    = IncrementIndex
    | InsertIndex   Int (Maybe Int)
    | InsertLowlink Int (Maybe Int)
    | Push Int
    | Pop
    | OutputSCC [Int]
    deriving (Eq, Show)

data EState = EState
    { current :: TState
    , history :: [Event]
    } deriving (Eq, Show)
    
whenM :: Monad m => m Bool -> m () -> m ()
whenM boolM block = boolM >>= \b -> if b then block else return ()

insert :: V.Vector a -> Int -> a -> V.Vector a
insert vector index value = vector V.// [(index, value)]

apply :: Event -> TState -> TState
apply event state = case event of
    IncrementIndex    -> state { index    = index state + 1 }
    InsertIndex k v   -> state { indices  = insert (indices  state) k v }
    InsertLowlink k v -> state { lowlinks = insert (lowlinks state) k v }
    Push i            -> state
        { stack    = i : (stack state) 
        , stackSet = insert (stackSet state) i True
        }
    Pop               -> let i = head (stack state) in state
        { stack    = tail (stack state)
        , stackSet = insert (stackSet state) i False
        , stackTop = i
        }
    OutputSCC scc     -> state { output = output state ++ [scc] }

initial = TState 0 [] falses undefined nothings nothings []
     where 
        size     = (fst $ M.findMax graph) + 1
        falses   = V.replicate size False
        nothings = V.replicate size Nothing

emit :: Event -> State EState ()
emit event = void . modify' $ \s -> s
    { current = apply event (current s) 
    , history = event : (history s)
    }

incrementIndex    = emit   IncrementIndex
insertIndex   k v = emit $ InsertIndex k v
insertLowlink k v = emit $ InsertLowlink k v
push i            = emit $ Push i
pop               = emit   Pop
outputSCC scc     = emit $ OutputSCC scc

query accessor i = (V.! i) . accessor <$> gets current

In [4]:
tarjan :: M.IntMap [Int] -> ([[Int]], EState)
tarjan graph = flip runState (EState initial []) $ do
    for_ (M.keys graph) $ \v ->
        whenM ((Nothing==) <$> query indices v) $
            strongConnect v graph
    output <$> gets current

strongConnect :: Int -> M.IntMap [Int] -> State EState ()
strongConnect v graph = do
    incrementIndex
    i <- index <$> gets current
    insertIndex   v (Just i)
    insertLowlink v (Just i)
    push v
    for_ (graph M.! v) $ \w -> do
        i <- query indices w
        case i of
            Nothing -> do
                strongConnect w graph
                insertLowlink v =<< (min <$> query lowlinks v <*> query lowlinks w)
            Just{}  -> whenM (query stackSet w) $ do
                insertLowlink v =<< (min <$> query lowlinks v <*> query indices  w)
    whenM ((==) <$> query lowlinks v <*> query indices v) $
        outputSCC =<< addSCC v []

addSCC :: Int -> [Int] -> State EState [Int]
addSCC v scc = do
    pop
    w <- stackTop <$> gets current
    let scc' = w:scc
    if w == v
        then return scc'
        else addSCC v scc'

In [5]:
complete = tarjan graph
void $ traverse print (reverse . history . snd $ complete)

IncrementIndex
InsertIndex 1 (Just 1)
InsertLowlink 1 (Just 1)
Push 1
IncrementIndex
InsertIndex 2 (Just 2)
InsertLowlink 2 (Just 2)
Push 2
IncrementIndex
InsertIndex 3 (Just 3)
InsertLowlink 3 (Just 3)
Push 3
InsertLowlink 3 (Just 1)
InsertLowlink 2 (Just 1)
InsertLowlink 1 (Just 1)
Pop
Pop
Pop
OutputSCC [1,2,3]
IncrementIndex
InsertIndex 4 (Just 4)
InsertLowlink 4 (Just 4)
Push 4
IncrementIndex
InsertIndex 5 (Just 5)
InsertLowlink 5 (Just 5)
Push 5
InsertLowlink 5 (Just 4)
IncrementIndex
InsertIndex 6 (Just 6)
InsertLowlink 6 (Just 6)
Push 6
IncrementIndex
InsertIndex 7 (Just 7)
InsertLowlink 7 (Just 7)
Push 7
InsertLowlink 7 (Just 6)
InsertLowlink 6 (Just 6)
Pop
Pop
OutputSCC [6,7]
InsertLowlink 5 (Just 4)
InsertLowlink 4 (Just 4)
Pop
Pop
OutputSCC [4,5]
IncrementIndex
InsertIndex 8 (Just 8)
InsertLowlink 8 (Just 8)
Push 8
InsertLowlink 8 (Just 8)
Pop
OutputSCC [8]